In [ ]:
# Cell 1: imports and load cleaned file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Settings for nicer plots
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,5)
sns.set(style="whitegrid")

# Path to cleaned file (edit if your file has a different name)
CLEAN_PATH = Path("../data/processed/cleaned_retail.csv")

# Load dataset (try to parse common date column names)
print("Loading:", CLEAN_PATH.resolve())
df = pd.read_csv(CLEAN_PATH, low_memory=False)

# if invoice date column is string-like, try to parse it explicitly
date_candidates = [c for c in df.columns if 'date' in c.lower()]
print("Date columns found:", date_candidates)
if date_candidates:
    df[date_candidates[0]] = pd.to_datetime(df[date_candidates[0]], errors='coerce')
    # normalize to a standard name for later cells
    df = df.rename(columns={date_candidates[0]: 'invoice_date'})

# show a brief preview
print("\nPreview (first 5 rows):")
display(df.head())


In [1]:
# Cell 1: imports and load cleaned file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Settings for nicer plots
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,5)
sns.set(style="whitegrid")

# Path to cleaned file (edit if your file has a different name)
CLEAN_PATH = Path("../data/processed/cleaned_retail.csv")

# Load dataset (try to parse common date column names)
print("Loading:", CLEAN_PATH.resolve())
df = pd.read_csv(CLEAN_PATH, low_memory=False)

# if invoice date column is string-like, try to parse it explicitly
date_candidates = [c for c in df.columns if 'date' in c.lower()]
print("Date columns found:", date_candidates)
if date_candidates:
    df[date_candidates[0]] = pd.to_datetime(df[date_candidates[0]], errors='coerce')
    # normalize to a standard name for later cells
    df = df.rename(columns={date_candidates[0]: 'invoice_date'})

# show a brief preview
print("\nPreview (first 5 rows):")
display(df.head())



Loading: C:\Users\Jiya\FUTURE_DS_01\data\processed\cleaned_retail.csv
Date columns found: ['InvoiceDate']

Preview (first 5 rows):


,Invoice,StockCode,Description,Quantity,invoice_date,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [2]:
# Cell 2: shape, columns, data types, missing counts
print("Shape (rows, cols):", df.shape)
print("\nColumns:")
print(df.columns.tolist())

print("\nData types (sample):")
display(df.dtypes.head(12))

print("\nMissing values (top 15 columns):")
missing = df.isna().sum().sort_values(ascending=False)
display(missing.head(15))


Shape (rows, cols): (400916, 8)

Columns:
['Invoice', 'StockCode', 'Description', 'Quantity', 'invoice_date', 'Price', 'Customer ID', 'Country']

Data types (sample):


Invoice                  int64
StockCode               object
Description             object
Quantity                 int64
invoice_date    datetime64[ns]
Price                  float64
Customer ID            float64
Country                 object
dtype: object


Missing values (top 15 columns):


Invoice         0
StockCode       0
Description     0
Quantity        0
invoice_date    0
Price           0
Customer ID     0
Country         0
dtype: int64

In [3]:
# Cell 3: sanity checks
# Edit column names below if your columns are named differently
qty_col = next((c for c in df.columns if 'quantity' in c.lower()), None)
price_col = next((c for c in df.columns if 'price' in c.lower()), None)
invoice_col = next((c for c in df.columns if 'invoice' in c.lower()), None)
cust_col = next((c for c in df.columns if 'customer' in c.lower()), None)

print("Detected columns:")
print(" Quantity:", qty_col)
print(" Price   :", price_col)
print(" Invoice :", invoice_col)
print(" Customer:", cust_col)

# Basic numeric checks
if qty_col:
    print("\nQuantity - min, max, nulls:", df[qty_col].min(), df[qty_col].max(), df[qty_col].isna().sum())
if price_col:
    print("Price    - min, max, nulls:", df[price_col].min(), df[price_col].max(), df[price_col].isna().sum())

# Check for negative/zero values (should be none after cleaning)
if qty_col and price_col:
    neg_qty = (df[qty_col] <= 0).sum()
    neg_price = (df[price_col] <= 0).sum()
    print("\nRows with non-positive quantity:", neg_qty)
    print("Rows with non-positive price   :", neg_price)

# Missing customer ids
if cust_col:
    print("\nMissing Customer IDs:", df[cust_col].isna().sum())
else:
    print("\nCustomer column not detected.")
    
# Check invoice_date parsed
if 'invoice_date' in df.columns:
    print("\nInvoice date range:", df['invoice_date'].min(), "→", df['invoice_date'].max())
else:
    print("\ninvoice_date column not found — check date parsing in Cell 1.")


Detected columns:
 Quantity: Quantity
 Price   : Price
 Invoice : Invoice
 Customer: Customer ID

Quantity - min, max, nulls: 1 19152 0
Price    - min, max, nulls: 0.001 10953.5 0

Rows with non-positive quantity: 0
Rows with non-positive price   : 0

Missing Customer IDs: 0

Invoice date range: 2009-12-01 07:45:00 → 2010-12-09 20:01:00


In [4]:
# Cell 4: create standardized columns used later (safeguard - will not overwrite if present)
# Standard names we will use: invoice_no, stock_code, description, quantity, price, customer_id, country, invoice_date

def find_and_rename(df, candidates, newname):
    for c in df.columns:
        if c.lower() in candidates:
            df = df.rename(columns={c: newname})
            return df
    return df

# candidates are lowercase forms to check against column names
df = find_and_rename(df, ['invoice','invoiceno','invoice_no','invoice number'], 'invoice_no')
df = find_and_rename(df, ['stockcode','stock_code','stock code'], 'stock_code')
df = find_and_rename(df, ['description','desc','product_description'], 'description')
df = find_and_rename(df, ['quantity','qty'], 'quantity')
df = find_and_rename(df, ['price','unitprice','unit_price'], 'price')
df = find_and_rename(df, ['customer id','customerid','customer_id','cust_id'], 'customer_id')
df = find_and_rename(df, ['country'], 'country')
# invoice_date from prior cell will remain if set

# create totalprice if not present
if 'totalprice' not in df.columns and 'quantity' in df.columns and 'price' in df.columns:
    df['totalprice'] = df['quantity'] * df['price']

# create month_year for grouping
if 'invoice_date' in df.columns:
    df['month_year'] = df['invoice_date'].dt.to_period('M').astype(str)
    df['year'] = df['invoice_date'].dt.year
    df['month'] = df['invoice_date'].dt.month

print("Standard column names present now:", [c for c in ['invoice_no','stock_code','description','quantity','price','customer_id','country','invoice_date'] if c in df.columns])
display(df.head(3))


Standard column names present now: ['invoice_no', 'stock_code', 'description', 'quantity', 'price', 'customer_id', 'country', 'invoice_date']


,invoice_no,stock_code,description,quantity,invoice_date,price,customer_id,country,totalprice,month_year,year,month
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4,2009-12,2009,12
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12,2009,12
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0,2009-12,2009,12


In [5]:
# Cell 5: summary stats and top KPIs
total_sales = df['totalprice'].sum() if 'totalprice' in df.columns else np.nan
total_orders = df['invoice_no'].nunique() if 'invoice_no' in df.columns else df.shape[0]
unique_customers = df['customer_id'].nunique() if 'customer_id' in df.columns else np.nan
unique_products = df['stock_code'].nunique() if 'stock_code' in df.columns else np.nan

print("Total sales (sum):", round(total_sales,2))
print("Total orders       :", total_orders)
print("Unique customers   :", unique_customers)
print("Unique products    :", unique_products)


Total sales (sum): 8798233.74
Total orders       : 19213
Unique customers   : 4312
Unique products    : 4017
